In [71]:
import speech_recognition as sr
import pyttsx3
import datetime
import wikipedia
import requests
import urllib.parse
import webbrowser
import time
import random
from translate import Translator
from langdetect import detect


"""Observações de desenvolvedor:
o que foi dito irá ser transformado em letra MINÚSCULA pois esse é o padrão pra que ela realize o comando desejado"""

nome_assistente = "lia"  # Defina o nome da sua assistente aqui
parada = "pare" #defina o comando de parada

#lista de piadas:
piadas = [
            "Por que a mulher do Hulk divorciou-se dele? Porque ela queria um homem mais maduro.",
            "Qual é o contrário de volátil? Vem cá sobrinho!",
            "Você conhece a piada do pônei? Pô nei eu...",
            "Por que o livro de matemática cometeu suicídio? Porque tinha muitos problemas.",
        ] 

palavroes = ['retardada','idiota','caralho','cu','cuzona','filha da puta','filho da puta','desgraçado','desgraçada',
             'merda','piranha','porra','bosta']


In [72]:
#### Funções para converter hora em algarismos para hora em texto, necessário para o comando de voz que diz as horas
def numero_para_palavra(numero):
    unidades = ["zero", "um", "dois", "três", "quatro", "cinco", "seis", "sete", "oito", "nove",
                "dez", "onze", "doze", "treze", "quatorze", "quinze", "dezesseis", "dezessete",
                "dezoito", "dezenove"]
    dezenas = ["", "", "vinte", "trinta", "quarenta", "cinquenta", "sessenta", "setenta", "oitenta", "noventa"]
    if 0 <= numero < 20:
        return unidades[numero]
    elif 20 <= numero < 100:
        dezena, unidade = divmod(numero, 10)
        return dezenas[dezena] + (" e " + unidades[unidade] if unidade != 0 else "")
    else:
        return str(numero)
def hora_para_texto(hora):
    horas, minutos = hora.split(':')
    horas_texto = numero_para_palavra(int(horas))
    minutos_texto = numero_para_palavra(int(minutos))
    return horas_texto + ' e ' + minutos_texto
####

def comando_voz_usuario():  # Saída de áudio
    comando = comando_sem_nome
    if 'horas' in comando:  # Retorna a hora
        hora_atual = datetime.datetime.now().strftime('%H:%M')
        hora_texto = hora_para_texto(hora_atual)
        maquina.say('Agora são ' + hora_texto)
        maquina.runAndWait()

    elif 'procure por' in comando: #Retorna informações dadas pelo resumo do Wikipédia
        procurar = comando.replace('procure por', '')
        wikipedia.set_lang('pt')
        resultado = wikipedia.summary(procurar, 2)
        print(resultado)
        maquina.say(resultado)
        maquina.runAndWait()

    elif 'toque' in comando:  # Reproduz primeiro vídeo no YouTube
        musica = comando.replace('toque', '')
        pesquisa = musica + ' música'  # Adiciona "música" à pesquisa para melhorar os resultados
        pesquisa_encoded = urllib.parse.quote(pesquisa)
        url = "https://www.youtube.com/results?search_query={}".format(pesquisa_encoded)
        time.sleep(3)
        response = requests.get(url)
        video_id = None
        if response.ok:
            start = response.text.find('{"videoId":"') + len('{"videoId":"')
            end = response.text.find('","webPageType"')
            video_id = response.text[start:end]
        if video_id:
            youtube_url = "https://www.youtube.com/watch?v={}".format(video_id)
            maquina.say('Tocando música')
            webbrowser.open(youtube_url)
            maquina.runAndWait()
        else:
            maquina.say('Nenhum vídeo encontrado')
            maquina.runAndWait()

    elif 'procure vídeos' in comando:  # Procura qualquer vídeo no YouTube
        musica = comando.replace('procure vídeos', '')
        url = 'https://www.youtube.com/results?search_query={}'.format(musica)
        maquina.say('Abrindo resultados da pesquisa')
        webbrowser.open(url)
        maquina.runAndWait()
    
    elif 'procure imagens de' in comando:  # Procura imagens no Google
        termo_pesquisa = comando.replace('procure imagens de', '')
        url = 'https://www.google.com.br/search?q={}&gl=br&tbm=isch&sxsrf=AB5stBhN31bua67Jlg6trgh_zAHiFXDfcA%3A1690422583557&source=hp&biw=1280&bih=907&ei=N83BZKOEIKTK1sQP7p-EwAw&iflsig=AD69kcEAAAAAZMHbRwR-O0rgx2I2QcSdNpnj-u4UmupA&oq=&gs_lp=EgNpbWciACoCCAAyBxAjGOoCGCcyBxAjGOoCGCcyBxAjGOoCGCcyBxAjGOoCGCcyBxAjGOoCGCcyBxAjGOoCGCcyBxAjGOoCGCcyBxAjGOoCGCcyBxAjGOoCGCcyBxAjGOoCGCdI2TJQAFgAcAF4AJABAJgBAKABAKoBALgBAcgBAIoCC2d3cy13aXotaW1nqAIK&sclient=img'.format(termo_pesquisa)
        maquina.say('Pesquisando imagens de ' + termo_pesquisa)
        webbrowser.open(url)
        maquina.runAndWait()

    elif 'pesquise por' in comando:  # Pesquisar qualquer coisa no Google
        termo_pesquisa = comando.replace('pesquise por', '')
        url = 'https://www.google.com/search?q={}'.format(termo_pesquisa)
        maquina.say('Pesquisando' + termo_pesquisa)
        webbrowser.open(url)
        maquina.runAndWait()

    elif 'abra meu e-mail' in comando:  # Abrindo e-mail
        url = 'https://mail.google.com/mail/?authuser=0'
        maquina.say('Abrindo e-mail')
        webbrowser.open(url)
        maquina.runAndWait()
        
    elif 'bem' in comando:
        maquina.say('Poderia estar melhor se tivesse passado em lógica...')
        maquina.runAndWait()

    elif 'ache' in comando:  # Pesquisando no google maps
        termo_pesquisa = comando.replace('ache', '')
        url = 'https://www.google.com.br/maps/search/{}/'.format(termo_pesquisa)
        maquina.say('Pesquisando no Google Maps'+termo_pesquisa)
        webbrowser.open(url)
        maquina.runAndWait()
        
    elif 'piada' in comando: #fazendo uma piada
        
        # Selecionar uma piada aleatória
        piada = random.choice(piadas)

        # Falar a piada
        maquina.say(piada)
        maquina.runAndWait()
    
        
    elif palavrao(comando) is True: #verificando se tem palavrão
        maquina.say("Por favor pare de usar palavras chulas, fale novamente com mais educação")
        maquina.runAndWait()
        
    elif 'traduza' in comando:
        comando = comando.replace('lia','')
        comando = comando.replace('traduza','')
        frase = traduzir(comando)
        
        if palavrao(frase) is True: #verificando se tem palavrão
            maquina.say("Você achou que eu ia falar um palavrão não é?! Mas eu não sou burra. Me dê outro comando")
            print("Você achou que eu ia falar um palavrão não é?! Mas eu não sou burra. Me dê outro comando")
            maquina.runAndWait()
        else:
            maquina.say(f"A frase traduzida é: {frase}")
            print(f"A frase traduzida é: {frase}")
            maquina.runAndWait()

In [73]:
#função pra verificar se tem palavrão:
def palavrao (comando):
    def palavrao1(palavroes):
        comando
        if palavroes in comando:
            return True
        else:
            return False
    
    resposta = list(map(palavrao1,palavroes))
    
    if True in resposta:
        return True
    else: 
        return False
    
###
#função pra operações matemáticas

    
###

#função pra tradução automática:

def traduzir(texto):
    idioma_detectado = detect(texto)
    translator = Translator(from_lang= str(idioma_detectado), to_lang='pt')
    resultado = translator.translate(texto)
    return resultado


In [74]:

def escutar():
    reconhecimento = sr.Recognizer()
    with sr.Microphone() as source:
        print("Aguardando comando...")
        reconhecimento.adjust_for_ambient_noise(source)  # Ajustar ruído ambiente
        audio = reconhecimento.listen(source)

    try:
        comando = reconhecimento.recognize_google(audio, language='pt-BR')
        print("Você disse: " + comando)
        return comando.lower()
    except sr.UnknownValueError:
        print("Não foi possível reconhecer o áudio.")
        return ""
    except sr.RequestError:
        print("Não foi possível acessar o serviço de reconhecimento de voz. Verifique sua conexão com a internet.")
        return ""


while True:
    maquina = pyttsx3.init()
    comando = escutar()
    if parada in comando:
        print(f"Vou descansar um pouco")
        maquina.say("Vou descansar um pouco")
        maquina.runAndWait()
        break
        
    elif nome_assistente in comando:
        print("Assistente ativada...")
        comando_sem_nome = comando.replace(nome_assistente, '').strip().lower()
        print("Comando: " + comando_sem_nome)
        comando_voz_usuario()


Aguardando comando...
Você disse: Lia traduza I'm fucking good
Assistente ativada...
Comando: traduza i'm fucking good
Você achou que eu ia falar um palavrão não é?! Mas eu não sou burra. Me dê outro comando
Aguardando comando...
Você disse: Lia traduza Cristiano Ronaldo is the best
Assistente ativada...
Comando: traduza cristiano ronaldo is the best
A frase traduzida é: Cristiano Ronaldo é o melhor
Aguardando comando...
Você disse: Lia pare
Vou descansar um pouco
